In [115]:
#! /usr/bin/python3
import numpy as np
from glob import glob
import csv
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2

In [174]:
%cd '/workspace'

/workspace


In [175]:
images = glob('{}/*/*_image.jpg'.format('trainval'))
def help(s):
  return s[-51:]
images = sorted(images, key=help)
len(images)

7573

In [176]:
labels = pd.read_csv('trainval_labels.csv')

In [177]:
labels.set_index('guid/image',inplace=True)
D = labels.to_dict()

In [178]:
len(D['label'])

7573

In [151]:
count = 0
for idx in range(len(images)):
    name = images[idx]

    L = name.split('/')
    # print(L)
    guid = L[1]
    id = L[2][:4]
    if f'{guid}/{id}' in D['label']:
        count += 1
count, len(images)

(7573, 7573)

In [179]:
count = 0
tups = []
for idx in range(len(images)):
    name = images[idx]

    L = name.split('/')
    guid = L[1]
    id = L[2][:4]
    tups.append((images[idx], D['label'][f'{guid}/{id}']))


In [181]:
import pickle
with open('tups.pkl', 'wb') as f:
    pickle.dump(tups, f)

In [153]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [188]:
class ImageDataloader(Dataset):
    """
    Dataloader for Inference.
    """
    def __init__(self, image_paths, label_dict, target_size=256):

        self.img_paths = image_paths
        self.label_dict = label_dict
        self.target_size = target_size
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
                transforms.ToPILImage(),transforms.Resize((target_size,target_size)),transforms.ToTensor()
        ])

    def __getitem__(self, idx):
        """
        __getitem__ for inference
        :param idx: Index of the image
        :return: img_np is a numpy RGB-image of shape H x W x C with pixel values in range 0-255.
        And img_tor is a torch tensor, RGB, C x H x W in shape and normalized.
        """
        img = cv2.imread(self.img_paths[idx])
        name = self.img_paths[idx]

        L = name.split('/')
        guid = L[1]
        id = L[2][:4]

        if self.label_dict:
            label = self.label_dict[f'{guid}/{id}']
        else:
            # test
            label = -1

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Pad images to target size
        img_tor = self.transform(img)
        # img_tor = img_np.astype(np.float32)
        img_tor = img_tor / 255.0
        # img_tor = self.normalize(img_tor)

        # one_hot = torch.nn.functional.one_hot(torch.tensor(label), num_classes=3)
        return img_tor, label

    def __len__(self):
      return len(self.img_paths)

In [155]:
from torch.utils.data import random_split

batch_size = 128
val_size = 1000
train_size = len(images) - val_size 

train_data,val_data = random_split(images,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

train_data = ImageDataloader(train_data, D['label'])
val_data = ImageDataloader(val_data, D['label'])

#load the train and validation into batches.
train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)

Length of Train Data : 6573
Length of Validation Data : 1000


In [156]:
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

def accuracy(outputs, labels):
    _, preds = torch.max(outputs.to(device), dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        print('epoch', epoch)
        
        model.train()
        train_losses = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images.to(device))                  # Generate predictions
        loss = F.cross_entropy(out.to(device), labels.to(device)) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images.to(device))                    # Generate predictions
        loss = F.cross_entropy(out.to(device), labels.to(device))   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [186]:
class CarClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(64 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256 ,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(65536,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,3)
        )
    
    def forward(self, xb):
        return self.network(xb)

model = CarClassification()
model.to(device)

CarClassification(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): ReLU()
    (17): Conv2d(256, 256, kernel_size=(3, 3), stride=

In [160]:
print(model)

CarClassification(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Flatten(start_dim=1, end_dim=-1)
    (11): Linear(in_features=65536, out_features=1024, bias=True)
    (12): ReLU()
    (13): Linear(in_features=1024, out_features=512, bias=True)
    (14): ReLU()
    (15): Linear(in_features=512, out_features=3, bias=True)
  )
)


In [128]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

num_epochs = 30
opt_func = torch.optim.Adam
lr = 0.001
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

epoch 0


  0%|          | 0/52 [00:00<?, ?it/s]

tensor([2, 0, 2, 2, 1, 2, 1, 0, 0, 1, 1, 1, 0, 1, 2, 1, 0, 1, 0, 1, 1, 1, 2, 0,
        1, 2, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        0, 1, 1, 0, 1, 2, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 0, 2, 1,
        1, 2, 0, 0, 0, 0, 1, 1, 0, 1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 2, 0,
        1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 2, 2, 0, 2, 1, 2, 1, 0, 0, 1,
        1, 1, 1, 1, 2, 2, 0, 0])
tensor([[ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.0049,  0.0363, -0.0033],
        [ 0.

  2%|▏         | 1/52 [00:11<09:27, 11.14s/it]

tensor([2, 2, 1, 0, 1, 1, 0, 2, 0, 1, 2, 0, 1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1, 2,
        0, 2, 1, 2, 1, 0, 1, 1, 1, 0, 0, 1, 2, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 1,
        2, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1, 0, 1, 2, 0, 2, 2, 1, 0, 1, 0, 1, 2, 2,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 1, 0, 0, 2, 0, 1,
        1, 1, 0, 1, 1, 2, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 0, 1])
tensor([[ 0.5836,  0.6003, -0.8885],
        [ 0.5836,  0.6003, -0.8885],
        [ 0.5835,  0.6002, -0.8884],
        [ 0.5837,  0.6004, -0.8886],
        [ 0.5835,  0.6002, -0.8884],
        [ 0.5835,  0.6002, -0.8884],
        [ 0.5837,  0.6004, -0.8886],
        [ 0.5837,  0.6004, -0.8886],
        [ 0.5835,  0.6002, -0.8883],
        [ 0.5837,  0.6004, -0.8886],
        [ 0.5836,  0.6003, -0.8885],
        [ 0.5837,  0.6004, -0.8886],
        [ 0.5835,  0.6003, -0.8884],
        [ 0.5836,  0.6003, -0.8884],
        [ 0.5837,  0.6004, -0.8886],
        [ 0.

  2%|▏         | 1/52 [00:17<15:13, 17.91s/it]


KeyboardInterrupt: 

# Testing

In [182]:
#! /usr/bin/python3
import numpy as np
from glob import glob
import csv

test_images = glob('{}/*/*_image.jpg'.format('test'))

In [193]:
batch_size = 56

print(f"Length of Test Data : {len(test_images)}")

test_data = ImageDataloader(test_images, None)

#load the train and validation into batches.
test_dl = DataLoader(test_data, batch_size, shuffle = False, num_workers = 1, pin_memory = True)

Length of Test Data : 2631


In [196]:
model = CarClassification()
model.load_state_dict(torch.load('7.ckpt'))
device='cpu'
model.to(device)
model.eval()
all_preds = []
with torch.no_grad():
  for i, test_data in tqdm(enumerate(test_dl), total=len(test_dl)):
      images, labels = test_data
      out = model(images.to(device))
      _, preds = torch.max(out.to(device), dim=1)
      all_preds.extend(preds)

100%|██████████| 47/47 [05:51<00:00,  7.47s/it]


In [172]:
next(model.parameters()).is_cuda

True

In [197]:
len(all_preds)

2631

In [198]:
test_images
names = []
for name in test_images:
  L = name.split('/')
  guid = L[1]
  id = L[2][:4]
  names.append(f'{guid}/{id}')

In [199]:
all_preds[0].int()

tensor(1, dtype=torch.int32)

In [200]:
with open('8_epoch_results.csv', 'w') as f:
  for i in range(len(names)):
    f.write(f'{names[i]},{int(all_preds[i])}\n')